In [ ]:
'''
TODO:
- label manually for instances of class 3
'''

# Approach 2 - method 2 - step2 (Extract Features)
- Precisely crop the anomaly from the detections by syncing the subtrace before and after the anomaly w.r.t ref_samples
- to keep the lenght of feature vector same, we pad the features with trailing zeros to get length of 500 (max length of detection)
- The feature extraction is the dependent on the corresponding normal behaviour subtrace
- We tested this approach across all applications


In [ ]:
import json
import os
import sys
sys.path.append('../')  ### to detect libraries in the parent directory
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from collections import defaultdict
from libraries.utils import *
from libraries.exeint import exeInt
import plotly.express as px
from statistics import mode

# ############ configuration - trace ################
# ############################################


CODE = 'mamba2'       ### application (code)       ###  'theft_protection', 'mamba2', 'lora_ducy'
BEHAVIOUR_FAULTY = 'faulty_data'            ### normal, faulty_data
BEHAVIOUR_NORMAL = 'normal'            ### normal, faulty_data
THREAD = 'single'           ### single, multi
VER = 4                     ### format of data collection
WINDOW = 500                 ### window size for subsequence
SUBSEQ =  'diag_subseq'        # 'diag_subseq' , diag_subseq_multi       ### subsequence type, diag_subseq, subseq


base_dir = '../../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
normalbase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_NORMAL}'
faultybase_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR_FAULTY}'

print(normalbase_path)
print(faultybase_path)


################# configuration - diag ################
IS_VAR_WINDOW = False             ### True, False; wether to use variable window size or not

#####################################################


ref_samples_basepath = os.path.join(normalbase_path, f'diag_refsamples{WINDOW}')
ref_var_samples_basepath = os.path.join(normalbase_path, 'diag_var_refsamples')
diag_subseq_basepath = os.path.join(faultybase_path, f'{SUBSEQ}/subseq')
diag_el_basepath = os.path.join(faultybase_path, f'{SUBSEQ}/el')
subseq_label_basepath = os.path.join(diag_subseq_basepath, 'subseq_labels')
test_labels_basepath = os.path.join(faultybase_path, 'labels')


print('ref_samples_path:\n', ref_samples_basepath)
print('ref_var_samples_path:\n', ref_var_samples_basepath)
print('diag_subseq_path:\n', diag_subseq_basepath)

######### get paths #######################
ref_samples_path = [os.path.join(ref_samples_basepath, x) for x in os.listdir(ref_samples_basepath)]
# ref_var_samples_path = [os.path.join(ref_var_samples_basepath, x) for x in os.listdir(ref_var_samples_basepath)]   

train_varlist_path = os.listdir(normalbase_path)
train_varlist_path = [os.path.join(normalbase_path, x) for x in train_varlist_path if 'varlist' in x]

######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(faultybase_path)

test_subseq_path = [os.path.join(diag_subseq_basepath, x) for x in os.listdir(diag_subseq_basepath)]
test_el_path = [os.path.join(diag_el_basepath, x) for x in os.listdir(diag_el_basepath)]
test_labels_path = [os.path.join(subseq_label_basepath, x) for x in os.listdir(subseq_label_basepath)]
eval_labels_path = [os.path.join(test_labels_basepath, x) for x in os.listdir(test_labels_basepath)]


# ### remove.Ds_store from all lists
train_varlist_path = [x for x in train_varlist_path if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]
ref_samples_path = [x for x in ref_samples_path if '.DS_Store' not in x]
# ref_var_samples_path = [x for x in ref_var_samples_path if '.DS_Store' not in x]
test_subseq_path = [x for x in test_subseq_path if '.DS_Store' not in x if '.json' in x]
test_feature_path = [x for x in test_el_path if '.DS_Store' not in x]
test_labels_path = [x for x in test_labels_path if '.DS_Store' not in x]
eval_labels_path = [x for x in eval_labels_path if '.DS_Store' not in x]

varlist_path.sort()

# print(paths_log)
# print(paths_traces)
# print(varlist_path)
# print(paths_label)

if IS_VAR_WINDOW:
    # train_data_path = ref_var_samples_path
    raise ValueError('Ref samples for variable window missing')
else:
    train_data_path = ref_samples_path

test_data_path = test_subseq_path

# print('train_data:', train_data_path)
print(len(train_data_path))
# print('test_data:\n', test_data_path)
print(len(test_data_path))
print('test_labels:\n', test_labels_path)
print('eval_labels:\n', eval_labels_path)



In [ ]:
#### Load diag_sepAP2_m2

path_diag_sepAP2_m2 = os.path.join(faultybase_path, 'diag_sepAP2_m2/subseq')
path_diag_sepAP2_m2_feature = os.path.join(faultybase_path, 'diag_sepAP2_m2/feature')
path_labels_diag_sepAP2_m2 = os.path.join(path_diag_sepAP2_m2, 'subseq_labels')
print('path_diag_sepAP2_m2:', path_diag_sepAP2_m2)
print('path_labels_diag_sepAP2_m2:', path_labels_diag_sepAP2_m2)

files_sepap2 = os.listdir(path_diag_sepAP2_m2)
files_sepap2 = [os.path.join(path_diag_sepAP2_m2, x) for x in files_sepap2 if '.DS_Store' not in x]    ### remove .DS_Store
files_sepap2 = [x for x in files_sepap2 if os.path.isfile(x)]
feature_sepap2 = os.listdir(path_diag_sepAP2_m2_feature)
feature_sepap2 = [os.path.join(path_diag_sepAP2_m2_feature, x) for x in feature_sepap2 if '.DS_Store' not in x]    ### remove .DS_Store


labels_sepap2 = os.listdir(path_labels_diag_sepAP2_m2)
labels_sepap2 = [os.path.join(path_labels_diag_sepAP2_m2, x) for x in labels_sepap2 if '.DS_Store' not in x]    ### remove .DS_Store
labels_sepap2 = [x for x in labels_sepap2 if os.path.isfile(x)]


In [ ]:
##### Get file names for desired class
### class = 0, 1, 2; -1 for all classes
CLASS = 2

#### Load labels for diag_sepAP2_m2
labels_dict = read_json(labels_sepap2[0])
print('labels_dict:', labels_dict)


key_names = list(labels_dict.keys())
# print('key_names:', key_names)

sel_ap2_files = []
sel_ap2_classes = []
sel_ap2_features = []
for km in key_names:
    print('km:', km)
    _class = labels_dict[km][0]
    print('class:', _class)

    if _class == CLASS or CLASS == -1:
        _file_name = os.path.join(path_diag_sepAP2_m2, km+'.json')
        _feature_name = os.path.join(path_diag_sepAP2_m2_feature, km+'.json')
        print('file:', _file_name)
        ############# check if file exists or not
        if os.path.isfile(_file_name):
            print('file exists')
            sel_ap2_files.append(_file_name)
            sel_ap2_classes.append(_class)
            sel_ap2_features.append(_feature_name)
        else:
           raise ValueError('File not found:', _file_name)



In [ ]:
len(sel_ap2_features), #len(sel_ap2_files), len(sel_ap2_classes)

In [ ]:
if CODE == 'theft_protection':
        EVENT_MEDIAN = 22
        WINDOW = round(2 * EVENT_MEDIAN)    ### round(2 * median event range)
        SLIDING_WINDOW = round(EVENT_MEDIAN * 1.5)    ### round( 0,4 * WINDOW)
        # SLIDING_WINDOW = round(WINDOW - 25)   
        print('WINDOW:', WINDOW)
        print('SLIDING_WINDOW:', SLIDING_WINDOW) 
elif CODE == 'mamba2':
    EVENT_MEDIAN = 57
    WINDOW = round(2 * EVENT_MEDIAN)  ### round(1,8 * median event range)
    SLIDING_WINDOW = round(EVENT_MEDIAN * 1.5)    ### round( 0,7 * WINDOW)
    # SLIDING_WINDOW = round(WINDOW - 20)    
    print('WINDOW:', WINDOW)
    print('SLIDING_WINDOW:', SLIDING_WINDOW)

#### Load files
sel_subseq = []
sel_features = []
for file, feature, clas in zip(sel_ap2_files, sel_ap2_features, sel_ap2_classes):
    print('file:', file)
    det_subseq = read_json(file)
    print('subseq:', det_subseq)
    sel_subseq.append(det_subseq)

    print('classes:', clas)
    det_feat = read_json(feature)
    det_feat = np.array(det_feat)
    print('feature:', det_feat)

    ### pad features
    len_feat = det_feat.shape[0]
    pad_num = 500 - len_feat
    if pad_num > 0:
        pad_feat = np.pad(det_feat, (0, pad_num), 'constant', constant_values=(0))
    elif pad_num < 0:
        pad_feat = det_feat[:500]
        
    sel_features.append(pad_feat)
    
    print('')

    # break
sel_features = np.array(sel_features)
print('padded feature:', sel_features)

## Clustering

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Example variables
# padded_features = np.random.rand(x, 500)  # Replace with your actual feature matrix
# test_files = ["file1", "file2", ..., "fileX"]  # Replace with your actual file names
# test_class = ["label1", "label2", ..., "labelX"]  # Replace with your actual labels

# Normalize the features
scaler = StandardScaler()
padded_features_normalized = scaler.fit_transform(sel_features)

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=2, metric="euclidean")
dbscan.fit(padded_features_normalized)

# Get cluster labels
cluster_labels = dbscan.labels_

# Group file names and labels by cluster
clusters = {}
for i, cluster in enumerate(cluster_labels):
    if cluster not in clusters:
        clusters[cluster] = {"files": [], "labels": [], "features": []}
    clusters[cluster]["files"].append(sel_ap2_files[i])
    clusters[cluster]["labels"].append(sel_ap2_classes[i])
    clusters[cluster]["features"].append(sel_features[i]) 

# Print the clusters
for cluster_id, cluster_data in clusters.items():
    print(f"Cluster {cluster_id}:")
    print(f"  Number of files: {len(cluster_data['files'])}")
    

for cluster_id, cluster_data in clusters.items():
    print(f"Cluster {cluster_id}:")
    # print(f"  Files: {cluster_data['files']}")
    # print(f"  Labels: {cluster_data['labels']}")
    files = cluster_data['files']
    labels = cluster_data['labels']
    features = cluster_data['features']
    for file, label, feat in zip(files, labels, features):
        print(file)
        print(label)
        print(feat[:50])
        print('')
    print('')

In [ ]:
# Flatten the labels by joining them into a single string per instance
ground_truth = np.array(["_".join(map(str, str(labels))) for labels in sel_ap2_classes])

# Convert flattened labels to numeric format
unique_classes = list(set(ground_truth))  # Get unique class labels
class_to_int = {cls: idx for idx, cls in enumerate(unique_classes)}  # Map each class to an integer
ground_truth_numeric = np.array([class_to_int[cls] for cls in ground_truth])


In [ ]:
ground_truth_numeric

In [ ]:
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score

# Apply DBSCAN clustering
dbscan.fit(padded_features_normalized)

# Get cluster labels
labels = dbscan.labels_

# Filter out noise points (label -1 indicates noise in DBSCAN)
filtered_indices = labels != -1
print(filtered_indices)
filtered_labels = labels[filtered_indices]
filtered_ground_truth = ground_truth_numeric[filtered_indices]

# Evaluation Metrics
ari = adjusted_rand_score(filtered_ground_truth, filtered_labels)
nmi = normalized_mutual_info_score(filtered_ground_truth, filtered_labels)
silhouette = silhouette_score(padded_features_normalized[filtered_indices], filtered_labels)
homogeneity = homogeneity_score(filtered_ground_truth, filtered_labels)
completeness = completeness_score(filtered_ground_truth, filtered_labels)
v_measure = v_measure_score(filtered_ground_truth, filtered_labels)

# Print results
# print(f"Adjusted Rand Index (ARI): {ari:.4f}")
print(f"Normalized Mutual Information (NMI): {nmi:.4f}")
print(f"Silhouette Score: {silhouette:.4f}")
print(f"Homogeneity: {homogeneity:.4f}")
print(f"Completeness: {completeness:.4f}")
print(f"V-Measure: {v_measure:.4f}")